# Greenland hydrological basin delineation (ArcticDEM 500 m)

This notebook shows the full workflow used to derive **surface hydrological basins** for the Greenland Ice Sheet from the 500 m ArcticDEM product.

The goal is to produce a single, reproducible script that:

1. Configures paths and GRASS GIS.
2. Imports the ArcticDEM and an ice mask.
3. Preprocesses the DEM (optional smoothing / sink filling).
4. Runs `r.watershed` / `r.stream.*` to extract drainage basins.
5. Exports the resulting basins as a GeoTIFF for further analysis.


## 1. Prerequisites

Before running this notebook you need:

- A working **QGIS + GRASS** installation (paths in the next cell must be adapted to your system).
- The 500 m ArcticDEM mosaic over Greenland (here called `arcticDEM_500m_ice_sheet.tif`).
- A polygon **ice mask** (here the PROMICE 2022 ice mask).
- Enough disk space to hold intermediate GRASS maps and the final basin GeoTIFF.

All file system paths below are currently hard‑coded to a local Windows setup.  
You should modify them to point to your own data locations before running.


## 2. Configure paths and start a GRASS session

This cell:

- Defines all user paths and tunable parameters (stream threshold, output directory, etc.).
- Sets up a temporary GRASS location based on the ArcticDEM raster.
- Imports the DEM and the ice mask into GRASS.
- Applies the ice mask so that only ice‑covered regions are used for flow routing.


In [5]:
import os
import sys
import subprocess
from pathlib import Path

# =============================================================================
# USER SETTINGS
# =============================================================================
DEM = r"E:\Rasmus\DTU\Cryo\4DGreenland\DEM\arcticDEM_500m_ice_sheet.tif"
ice_mask = r"E:\Rasmus\DTU\Cryo\4DGreenland\Ice_mask\02-PROMICE-2022-IceMask-polygon.gpkg"
OUT = r"E:\Rasmus\DTU\Cryo\4DGreenland\Basins_serious\anna_code_s_thres_500"

QGIS_PREFIX = r"C:\Program Files\QGIS 3.40.11"
STREAM_THRESHOLD = 500

# =============================================================================
# GRASS ENVIRONMENT INIT
# =============================================================================
GISBASE = fr"{QGIS_PREFIX}\apps\grass\grass84"
os.environ.update({
    "GISBASE": GISBASE,
    "GRASS_PYTHON": sys.executable,
    "PROJ_LIB": fr"{QGIS_PREFIX}\share\proj",
    "GDAL_DATA": fr"{QGIS_PREFIX}\share\gdal",
})

os.environ["PATH"] = os.pathsep.join([
    fr"{GISBASE}\bin",
    fr"{GISBASE}\extrabin",
    fr"{QGIS_PREFIX}\bin",
    fr"{QGIS_PREFIX}\apps\Qt5\bin",
    os.environ["PATH"],
])

sys.path.insert(0, fr"{GISBASE}\etc\python")

import grass.script as gs
import grass.script.setup as gsetup

Path(OUT).mkdir(parents=True, exist_ok=True)

# =============================================================================
# GRASS SESSION HELPERS
# =============================================================================
def start_grass_from_raster(raster_path, location="dem_loc", mapset="PERMANENT"):
    """Start GRASS in ~/Documents/grassdata based on a DEM."""
    gisdbase = Path.home() / "Documents" / "grassdata"
    gisdbase.mkdir(parents=True, exist_ok=True)

    loc_path = gisdbase / location
    mapset_path = loc_path / mapset

    if not loc_path.exists():
        print(f"📁 Creating new GRASS location: {loc_path}")
        gs.core.create_location(
            dbase=str(gisdbase),
            location=location,
            filename=raster_path,
            overwrite=True
        )
    else:
        print(f"✅ Using existing GRASS location: {loc_path}")

    gsetup.init(str(mapset_path))
    print(f"🌿 GRASS session initialized in:\n   {mapset_path}\n")
    print(gs.read_command("g.gisenv"))

    return str(gisdbase), location, mapset


def import_dem_native(input_path, out_name="dem"):
    """Import or clone DEM to a native GRASS raster."""
    raster = input_path.replace("\\", "/")
    try:
        gs.run_command("r.in.gdal", input=raster, output=out_name, flags="o", overwrite=True)
        print(f"✓ r.in.gdal → {out_name}")
    except Exception:
        gs.run_command("r.external", input=raster, output=f"{out_name}_ext", flags="o", overwrite=True)
        gs.run_command("g.region", raster=f"{out_name}_ext")
        gs.mapcalc(f"{out_name} = {out_name}_ext * 1.0", overwrite=True)
        print(f"✓ r.external + clone → {out_name}")
    gs.run_command("g.region", raster=out_name)


from shutil import which
def have(module_name): return which(module_name) is not None


# =============================================================================
# START GRASS AND IMPORT DEM
# =============================================================================
GISDBASE, LOCATION, MAPSET = start_grass_from_raster(DEM)
import_dem_native(DEM, out_name="dem")
print(gs.read_command("g.version"))

# =============================================================================
# GRASS ADDON HANDLING
# =============================================================================
APPDATA = os.environ.get("APPDATA", str(Path.home()))
addon_base = Path(APPDATA) / "GRASS8" / "addons"
os.environ["GRASS_ADDON_BASE"] = str(addon_base)

addon_bin = addon_base / "bin"
addon_scripts = addon_base / "scripts"

os.environ["PATH"] = os.pathsep.join([
    str(addon_bin),
    str(addon_scripts),
    os.environ["PATH"]
])
os.environ["GRASS_ADDON_PATH"] = os.pathsep.join([str(addon_bin), str(addon_scripts)])

from grass.script.core import find_program
from grass.script.core import CalledModuleError

def ensure_grass_addon(module_name: str):
    if find_program(module_name) is None:
        gs.run_command("g.extension", extension=module_name, operation="add", flags="f")

    path = find_program(module_name)
    if path is None:
        inst = gs.read_command("g.extension", flags="l")
        raise RuntimeError(
            f"{module_name} not found after install.\n"
            f"GRASS_ADDON_BASE={addon_base}\n"
            f"PATH has bin/scripts? {addon_bin.exists()} / {addon_scripts.exists()}\n"
            f"Installed addons (truncated):\n{inst[:600]}"
        )
    print(f"✓ {module_name} at {path}")


ensure_grass_addon("r.stream.order")

print(gs.read_command("g.gisenv"))
print("ADDON_BASE:", os.environ["GRASS_ADDON_BASE"])
print("PATH contains addon_bin?", str(addon_bin) in os.environ["PATH"])
print("PATH contains addon_scripts?", str(addon_scripts) in os.environ["PATH"])
print("find_program(r.stream.order):", find_program("r.stream.order"))

# =============================================================================
# SECOND GRASS INIT (kept exactly as in your script, just deduped imports)
# =============================================================================
gisdbase = r"C:\Users\s174035\Documents\grassdata"
location = "dem_loc"
mapset   = "PERMANENT"

gs.core.create_location(
    dbase=gisdbase,
    location=location,
    filename=DEM,
    overwrite=True
)

gsetup.init(os.path.join(gisdbase, location, mapset))
print("GRASS session at:", os.path.join(gisdbase, location, mapset))

# =============================================================================
# HELPERS
# =============================================================================
def safe(expr):
    try:
        gs.run_command("r.mapcalc", expression=expr, overwrite=True)
    except CalledModuleError as e:
        raise RuntimeError(f"Mapcalc failed: {expr}\n{e}")

def count(map_):
    try:
        txt = gs.read_command("r.univar", flags="g", map=map_)
        for line in txt.splitlines():
            if line.startswith(("n=", "non_null_cells=")):
                return int(float(line.split("=")[1]))
    except:
        return 0

# --- ensure addons go to a writable place (Windows-safe default) ---
os.environ.setdefault("GRASS_ADDON_BASE", str(Path.home() / ".grass8" / "addons"))

def ensure_grass_addon(module_name: str):
    """Install a GRASS addon if missing."""
    if find_program(module_name) is None:
        # -f: overwrite if an older copy exists
        gs.run_command("g.extension", extension=module_name, operation="add", flags="f")
        if find_program(module_name) is None:
            raise RuntimeError(f"Failed to install GRASS addon: {module_name}")

# before calling r.stream.order:
ensure_grass_addon("r.stream.order")


gs.run_command("r.in.gdal", input=DEM, output="dem", overwrite=True)
gs.run_command("g.region", raster="dem", flags="p")
#---------- Maske ----------#
gs.run_command("g.region", raster="dem")

# --- Import and rasterize the ice mask polygon ---
gs.run_command("v.import", input=ice_mask.replace("\\","/"), output="ice_mask_vec", overwrite=True)

gs.run_command(
    "v.to.rast",
    input="ice_mask_vec",
    output="ice_mask_rast",
    use="val",
    value=1,
    overwrite=True
)

# --- Apply the raster mask ---
gs.run_command("r.mask", raster="ice_mask_rast", overwrite=True)
print("✓ Mask set from ice mask polygon")


✅ Using existing GRASS location: C:\Users\s174035\Documents\grassdata\dem_loc
🌿 GRASS session initialized in:
   C:\Users\s174035\Documents\grassdata\dem_loc\PERMANENT

GISDBASE='C:\Users\s174035\Documents\grassdata';
LOCATION_NAME='dem_loc';
MAPSET='PERMANENT';

✓ r.in.gdal → dem
GRASS 8.4.1 (2025)

✓ r.stream.order at False
GISDBASE='C:\Users\s174035\Documents\grassdata';
LOCATION_NAME='dem_loc';
MAPSET='PERMANENT';

ADDON_BASE: C:\Users\s174035\AppData\Roaming\GRASS8\addons
PATH contains addon_bin? True
PATH contains addon_scripts? True
find_program(r.stream.order): False
GRASS session at: C:\Users\s174035\Documents\grassdata\dem_loc\PERMANENT
✓ Mask set from ice mask polygon


## 3. Preprocess DEM and derive basins

This cell:

- Optionally smooths the DEM using `r.neighbors` to reduce small‑scale noise.
- Computes flow accumulation and flow direction using `r.watershed`.
- Extracts a stream network from the flow accumulation grid with `r.stream.extract`.
- Delineates drainage basins draining to each stream segment with `r.stream.basins`.
- Exports the final basin map as a GeoTIFF in the chosen output directory.


In [6]:
gs.run_command(
    "r.neighbors",
    input="dem",
    output="dem_smoothed_raw",
    method="average",
    size=3,
    memory=300,    # MB of RAM, lower to avoid crash
    overwrite=True
)

gs.run_command(
    "r.fillnulls",
    input="dem_smoothed_raw",
    output="dem_smoothed",
    method="bilinear",
    overwrite=True
)

# Flow direction (D8)
gs.run_command("r.watershed", elevation="dem_smoothed",
               accumulation="accum", drainage="flow_dir",
               overwrite=True)

# Extract streams
gs.run_command("r.stream.extract", elevation="dem_smoothed",
               direction="flow_dir", accumulation="accum",
               threshold=STREAM_THRESHOLD, stream_raster="streams",
               overwrite=True)

# Basins
gs.run_command("r.stream.basins", direction="flow_dir",
               stream_rast="streams", basins="basins",
               flags="l", overwrite=True)

# choose data type per raster (optional but nice)
dtype = {
    "dem":        "Float32",
    "accum":      "Float64",   # accumulation can be large; Float32 also OK
    "flow_dir":   "Int32",     # from r.fill.dir
    "streams":    "Int16",     # binary/int
    "basins":     "Int32",     # labeled basins
    # "strahler": "Int16",
}

exports = [
    ("dem",      fr"{OUT}\dem_hydro.tif"),
    ("accum",    fr"{OUT}\accum_hydro.tif"),     
    ("flow_dir", fr"{OUT}\flowdir_hydro.tif"),
    ("streams",  fr"{OUT}\streams_hydro.tif"),
    ("basins",   fr"{OUT}\basins_hydro.tif"),
    # ("strahler", fr"{OUT}\strahler_order.tif"),
]

for name, fn in exports:
    try:
        info = gs.read_command("r.info", map=name)
        if "min =" in info and "max =" in info:
            print(f"📤 Exporting {name} → {fn}")
            gs.run_command(
                "r.out.gdal",
                input=name,
                output=fn.replace("\\", "/"),
                format="GTiff",
                type=dtype.get(name, "Float32"),
                createopt="COMPRESS=LZW,TILED=YES,BIGTIFF=YES",
                nodata=-9999,
                overwrite=True
            )
        else:
            print(f"⚠️ Skipping {name}: no data detected.")
    except Exception as e:
        print(f"❌ Failed to export {name}: {e}")



📤 Exporting dem → E:\Rasmus\DTU\Cryo\4DGreenland\Basins_serious\prodem_24\dem_hydro.tif
📤 Exporting accum → E:\Rasmus\DTU\Cryo\4DGreenland\Basins_serious\prodem_24\accum_hydro.tif
📤 Exporting flow_dir → E:\Rasmus\DTU\Cryo\4DGreenland\Basins_serious\prodem_24\flowdir_hydro.tif
📤 Exporting streams → E:\Rasmus\DTU\Cryo\4DGreenland\Basins_serious\prodem_24\streams_hydro.tif
📤 Exporting basins → E:\Rasmus\DTU\Cryo\4DGreenland\Basins_serious\prodem_24\basins_hydro.tif


## 4. (Optional) Extra commands / scratch space

Use this cell for additional diagnostics, QA plots, or to tweak parameters and re‑run parts of the workflow.
